In [1]:
include("model.jl")
include("Parser.jl")
using JuMP

## Data processing

In [2]:
n, m, x_VC, y_VC, x_loc, y_loc, Q, C, q, f, p, tc, B, R, localities_with_high_priorities, M = parse_instance("./Instances/coord20-5-1_processed.dat")


(20, 5, [6.0, 19.0, 37.0, 35.0, 5.0], [7.0, 44.0, 23.0, 6.0, 8.0], [20.0, 8.0, 29.0, 18.0, 19.0, 31.0, 38.0, 33.0, 2.0, 1.0, 26.0, 20.0, 15.0, 20.0, 17.0, 15.0, 5.0, 13.0, 38.0, 9.0], [35.0, 31.0, 43.0, 39.0, 47.0, 24.0, 50.0, 21.0, 27.0, 12.0, 20.0, 33.0, 46.0, 26.0, 19.0, 12.0, 30.0, 40.0, 5.0, 40.0], 70, [140, 140, 140, 140, 140], [17, 18, 13, 19, 12, 18, 13, 13, 17, 20, 16, 18, 15, 11, 18, 16, 15, 15, 15, 16], [10841, 11961, 6091, 7570, 7497], 1000, 0, 15549.245317275838, [13.958791596349233, 12.114607178090317, 9.951277348331352, 12.467724566948226, 14.326988158440654], [12, 5, 15], 5)

In [3]:
# Create the distance matrix
distance_matrix = zeros(Float64, m, n)

for i in 1:m 
    for j in 1:n
        distance_matrix[i, j] = sqrt((x_VC[i] - x_loc[j])^2 + (y_VC[i] - y_loc[j])^2)
    end
end


# Create reachability matrix a
A = zeros(Int, n, n)


for i in 1:m
    for j in 1:n
        if distance_matrix[i, j] <= R[i]
            A[i, j] = 1
        end
    end
end

In [4]:
D = zeros(Float64, n+m, n+m)
#The n first rows/cols of D relate to the localities, the m last to the VC
#In other words, calling D[i,j] for i in I, j in J corresponds to call D[i+n,j]
for i in 1:m+n
    for j in 1:m+n
        if i <= n && j <= n
            # Both indices in `loc`
            D[i, j] = sqrt((x_loc[i] - x_loc[j])^2 + (y_loc[i] - y_loc[j])^2)
        elseif i > n && j > n
            # Both indices in `VC`
            D[i, j] = sqrt((x_VC[i - n] - x_VC[j - n])^2 + (y_VC[i - n] - y_VC[j - n])^2)
        elseif i > n && j <= n
            # i in `VC`, j in `loc`
            D[i, j] = sqrt((x_VC[i - n] - x_loc[j])^2 + (y_VC[i - n] - y_loc[j])^2)
        else
            # i in `loc`, j in `VC`
            D[i, j] = sqrt((x_loc[i] - x_VC[j - n])^2 + (y_loc[i] - y_VC[j - n])^2)
        end
    end
end

## Models Creation

### OptVax1

In [5]:
model_OptVax1 = solve_OptVax1(n, m, D, A, Q, C, q, f, p, B, R, localities_with_high_priorities, M)

obj_val = objective_value(model_OptVax1)
println("Objective Value: ", obj_val)

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+04]
  Cost   [1e+01, 2e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+04]
Presolving model
2762 rows, 2673 cols, 14864 nonzeros  0s
2681 rows, 2594 cols, 14248 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2681 rows
   2594 cols (2594 binary, 0 integer, 0 implied int., 0 continuous)
   14248 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   315             -inf                 inf        0      0      0         0     0.2s
         0       0         0   0.00%   265             -inf                 inf        0      0     10       270     0.2s
 C       0       0         0   0.00

### OptVax2

In [6]:
model_OptVax2 = solve_OptVax2(n, m, D, A, Q, C, q, f, p, B, R, localities_with_high_priorities, M)
obj_val = objective_value(model_OptVax2)
println("Objective Value: ", obj_val)

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-01, 1e+04]
  Cost   [1e+01, 2e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+04]
Presolving model
2763 rows, 2673 cols, 14888 nonzeros  0s
2682 rows, 2594 cols, 14271 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2682 rows
   2594 cols (2594 binary, 0 integer, 0 implied int., 0 continuous)
   14271 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   315             -inf                 inf        0      0      0         0     0.2s
 R       0       0         0   0.00%   265             241                9.96%        0      0      0        46     0.2s
 C       0       0         0   0.00